# Keras-GAN
This is a basic implementation of GAN inspired by [Keras-GAN](https://github.com/eriklindernoren/Keras-GAN)

Inspired by [GAN](https://github.com/eriklindernoren/Keras-GAN/blob/master/gan/gan.py),
[DCGAN](https://github.com/eriklindernoren/Keras-GAN/blob/master/dcgan/dcgan.py),
[DCGAN by fchaollet](https://colab.research.google.com/github/fchollet/deep-learning-with-python-notebooks/blob/master/8.5-introduction-to-gans.ipynb#scrollTo=WLCF7I8opN7i), [DCGAN from TF with eager](https://github.com/tensorflow/tensorflow/blob/r1.11/tensorflow/contrib/eager/python/examples/generative_examples/dcgan.ipynb)


GANs use cases:


*   Upscale images
*   Infer the next frames in videos
*   Transform simplistic drawings into photorealistic sceneries
*   



In [1]:
# Install Dependencies
!pip install tensorboardcolab

In [41]:
#List available Devices
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()
# Check Versions
import tensorflow
import keras
print(tensorflow.__version__)
print(tensorflow.keras.__version__)
print(keras.__version__)

1.12.0
2.1.6-tf
2.2.4


In [32]:
# Run Tensorboard
from tensorboardcolab import *
tbc=TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
http://dfa3d666.ngrok.io


In [0]:
# Import libraries
from __future__ import print_function, division

from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
import matplotlib.pyplot as plt
from tensorflow.python.keras import models


import sys
import os
import time

import numpy as np

The process is as follows:

  


In [46]:
class GAN:
  def __init__(self, image_path):
    device = self.best_available_device()
    self.image_path = image_path
    self.img_rows = 28
    self.img_cols = 28
    self.channels = 1
    self.img_shape = (self.img_rows, self.img_cols, self.channels)
    self.latent_dim = 100

    optimizer = Adam(0.0002, 0.5)

    # Build and compile the discriminator
    self.discriminator_cpu_gpu = self.build_discriminator()
    print('Sequential', isinstance(self.discriminator_cpu_gpu, Sequential))
    print('Model', isinstance(self.discriminator_cpu_gpu, Model))
    model = models.clone_model(self.discriminator_cpu_gpu)
    print('Sequential Cloned', isinstance(model, Sequential))
    print('Model Cloned', isinstance(model, Model))
    
    if device == 'tpu':
      self.discriminator = tf.contrib.tpu.keras_to_tpu_model(
          self.discriminator_cpu_gpu,
          strategy=tf.contrib.tpu.TPUDistributionStrategy(
              tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
          )
      )
    else:
      self.discriminator = self.discriminator_cpu_gpu
    
    self.discriminator.compile(loss='binary_crossentropy',
      optimizer=optimizer,
      metrics=['accuracy'])

    # Build the generator
    self.generator = self.build_generator()

    # The generator takes noise as input and generates imgs
    z = Input(shape=(self.latent_dim,))
    img = self.generator(z)

    # For the combined model we will only train the generator
    self.discriminator_frozen = Model(inputs=self.discriminator_cpu_gpu.inputs,
                      outputs=self.discriminator_cpu_gpu.outputs)
    self.discriminator_frozen.trainable = False

    # The discriminator takes generated images as input and determines validity
    validity = self.discriminator_frozen(img)

    # The combined model  (stacked generator and discriminator)
    # Trains the generator to fool the discriminator
    self.combined_cpu_gpu = Model(inputs=z, outputs=validity)
    
    if device == 'tpu':
      self.combined = tf.contrib.tpu.keras_to_tpu_model(
          self.combined_cpu_gpu,
          strategy=tf.contrib.tpu.TPUDistributionStrategy(
              tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
          )
      )
    else:
      self.combined = self.combined_cpu_gpu
      
    self.combined.compile(loss='binary_crossentropy',
      optimizer=optimizer,
      metrics=['accuracy'])


  def build_generator(self):

    model = Sequential()

    model.add(Dense(256, input_dim=self.latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(self.img_shape), activation='tanh'))
    model.add(Reshape(self.img_shape))


    noise = Input(shape=(self.latent_dim,))
    img = model(noise)

    return Model(inputs=noise, outputs=img)

  def build_discriminator(self):

    model = Sequential()

    model.add(Flatten(input_shape=self.img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))

    img = Input(shape=self.img_shape)
    validity = model(img)

    return Model(inputs=img, outputs=validity)

  def train(self, epochs, batch_size=128, sample_interval=50):
    if False:
      print('Generator Model:')
      self.discriminator.summary()
      print('Discriminator Model:')
      self.combined.summary()

    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()
    
    # Rescale -1 to 1
    X_train = X_train / 127.5 - 1.
    X_train = np.expand_dims(X_train, axis=3)

    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    # Setup tensorboard
    # Create the TensorBoard callback,
    # which we will drive manually
    now = time.time()
    tensorboard_d = TensorBoard(
      log_dir= f'./Graph/{now}/d',
      histogram_freq=0,
      batch_size=batch_size,
      write_graph=True,
      write_grads=True
    )
    tensorboard_g = TensorBoard(
      log_dir= f'./Graph/{now}/g',
      histogram_freq=0,
      batch_size=batch_size,
      write_graph=True,
      write_grads=True
    )
    
    tensorboard_d.set_model(self.discriminator)
    tensorboard_g.set_model(self.combined)
    
    for epoch in range(epochs):

      # ---------------------
      #  Train Discriminator
      # ---------------------

      # Generate a batch of new images
      noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
      gen_imgs = self.generator.predict(noise)
      
      # Select a random batch of images
      idx = np.random.randint(0, X_train.shape[0], batch_size)
      imgs = X_train[idx]


      # Train the discriminator
      d_loss_real = self.discriminator.train_on_batch(imgs, valid)
      d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

      # ---------------------
      #  Train Generator
      # ---------------------

      noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

      # Train the generator (to have the discriminator label samples as valid)
      g_loss = self.combined.train_on_batch(noise, valid)

      # Plot the progress
      # print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f, acc.: %.2f%]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[0], g_loss[1]))
      tensorboard_d.on_epoch_end(epoch, self.named_logs(self.discriminator, d_loss))
      tensorboard_g.on_epoch_end(epoch, self.named_logs(self.combined, g_loss))
      # If at save interval => save generated image samples
      if epoch % sample_interval == 0:
        print(f'epoch: {epoch}')
        self.sample_images(epoch)
    
    tensorboard_d.on_train_end(None)
    tensorboard_g.on_train_end(None)

  def sample_images(self, epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, self.latent_dim))
    gen_imgs = self.generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
      for j in range(c):
        axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
        axs[i,j].axis('off')
        cnt += 1
    fig.savefig(self.image_path + "%d.png" % epoch)
    plt.close()
  
  # Transform train_on_batch return value
  # to dict expected by on_batch_end callback
  def named_logs(self, model, logs):
    result = {}
    for metric_name, log_value in zip(model.metrics_names, logs):
      result[metric_name] = log_value
    return result

  def best_available_device(self):
    gpu_available = tf.test.is_gpu_available()
    tpu_available = 'COLAB_TPU_ADDR' in os.environ
    if tpu_available:
      return 'tpu'
    elif gpu_available:
      return 'gpu'
    else:
      return 'cpu'
    
if __name__ == '__main__':
  image_path = './imgs/'
  if not os.path.exists(image_path):
    os.makedirs(image_path)
  gan = GAN(image_path)
  gan.train(epochs=5001, batch_size=32, sample_interval=500)

Sequential False
Model True
Sequential Cloned False
Model Cloned True
INFO:tensorflow:Querying Tensorflow master (b'grpc://10.68.193.138:8470') for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 2386358207207873321)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 14191721723296006097)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 5771321573470959913)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 6329731057825049777)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 171798691

RuntimeError: ignored

# How GAN works
- for each epoch:
  1. generator.predict
  2. discriminator.train_on_batch
  3. combo_frozen_discriminator.train_on_batch 

- for each epoch:
  1.   Generator creates fake images
  2.   Discriminator learns how to distinguish these fake images with real images
  3.   Generator learns how to create better fake images using Discriminator guides ${\frac{p_{data}(x)}{  p_{model}(x)}}$

Generator generate images that discrimintator cannot distinguish => combo low score!


---


Theoretical perfect final state:
-   Have 100% accuracy for the generator - meaning the discriminator classifies all synthetic observations as real;
-   Have about 50% accuracy for the discriminator - meaning it cannot distingiush fake observations from real ones;
-   The synthetic observations are of good quality.



---

How they should evolve:
-   Discriminator: Start from ~50% goes to ~100% very fast and comes back to 50%
-   Generator: Starts from ~0% and goes to ~100% acc

In [0]:
# Pseduo Code, NOT A RUNNABLE CELL.
The process is as follows:
init()
    # Build and compile the discriminator
    discriminator = build_discriminator() {
      model = Sequential()
      model.add()
      img_generator_output = Input(shape=self.img_shape)
      validity = model(img_generator_output)
      return Model(inputs=img_generator_output, outputs=validity)
    }
    discriminator.compile()

    # Build the generator
    generator = build_generator() {
      model = Sequential()
      model.add()
      noise = Input(shape=(self.latent_dim,))
      img_generator_output = model(noise)
      return Model(inputs=noise, outputs=img_generator_output)
    }
     # The generator takes noise as input and generates imgs
    z_generator_input_noise = Input(shape=(self.latent_dim,))
    img_generator_output = generator(z_generator_input_noise)

    discriminator.trainable = False;
    validity_discriminator_output = self.discriminator_frozen(img_generator_output)

    self.combined = Model(inputs=z_generator_input_noise, outputs=validity_discriminator_output)


gan.train()
  # Load the dataset X_train
  # Rescale -1 to 1
  # set Adversarial Y_valid = np.ones and Y_fake = np.zeros
  for epoch in range(epochs):
    # Generate a batch of new images
    gen_imgs = self.generator.predict(noise)

    # ***Discriminator Training***
    # Select a random batch of images
    # Train the discriminator
    # Ali: Would it help if I combine the real_imgs and the gen_imgs and then do one train_on_batch.
      d_loss_real = self.discriminator.train_on_batch(real_imgs, Y_valid)
      d_loss_fake = self.discriminator.train_on_batch(gen_imgs, Y_fake)
      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    # Now the discriminator knows how to classify fake images from true ones

    # ***Generator Training***
    # Train the generator in combined (to have the discriminator label samples as Y_valid)
      g_loss = self.combined.train_on_batch(noise, Y_valid)

    # If at save interval => save generated image samples


In [1]:
!pip install tensorboardcolab
from tensorboardcolab import *
tbc=TensorBoardColab()

Using TensorFlow backend.


Wait for 8 seconds...
TensorBoard link:
http://b3806594.ngrok.io
